# 🏟️ Tactix: Train Pitch Keypoint Detector (YOLO-Pose)

This notebook is designed to train a YOLOv8/11/26 Pose model to detect football pitch keypoints.
It is optimized for Kaggle Kernels (GPU enabled), but can run locally on your Mac M3 Pro.

## 1. Setup Environment

In [ ]:
# 确保安装了 ultralytics
# 如果在本地 Mac 跑，去掉前面的感叹号，直接在终端装好就行
!pip install ultralytics

import ultralytics
from ultralytics import YOLO
import os
from IPython.display import display, Image

print(f"Ultralytics version: {ultralytics.__version__}")

# Check GPU (Kaggle/Linux)
# 如果是 Mac，这行可能会报错，因为 Mac 用的是 mps 不是 nvidia-smi，可以注释掉
!nvidia-smi

## 2. Dataset Configuration

**IMPORTANT:**
- If running on **Kaggle**: Upload your dataset first via 'Add Data'.
- If running on **Mac**: Change `DATASET_ROOT` to your local folder path (e.g., `/Users/Julian/Tactix/data`).

We will generate a `data.yaml` file pointing to the correct paths.

In [ ]:
# --- CONFIGURATION ---
# ⚠️如果是本地 Mac，请修改这里为你的实际路径！
# 例如: DATASET_ROOT = "/Users/Julian/Projects/Tactix/datasets/pitch_v1"
DATASET_ROOT = "/kaggle/input/football-pitch"

# Define the YAML content
yaml_content = f"""
path: {DATASET_ROOT}
train: train/images
val: valid/images
test: test/images

# Keypoints
kpt_shape: [27, 2] # [number of keypoints, dim]
flip_idx: []

# Classes
names:
  0: pitch
"""

with open("pitch_data.yaml", "w") as f:
    f.write(yaml_content)

print("✅ Created pitch_data.yaml")
# 打印看看内容对不对
!cat pitch_data.yaml

## 3. Train Model

We use **YOLO11x-pose** (Extra Large) for maximum accuracy.
*(Note: You can change this to `yolo26l-pose.pt` if available)*

In [ ]:
# Load model
# 这里你可以改成 "yolo26l-pose.pt" 如果你想试新模型
model_name = "yolo26x-pose.pt"

print(f"🚀 Loading {model_name}...")
model = YOLO(model_name)

# Train
results = model.train(
    data="pitch_data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,        # Kaggle 填 0 或 [0,1]。Mac 本地填 'mps'
    project="runs/train",
    name="pitch_yolo11x",
    patience=20,
    save=True
)

## 4. Evaluation & Visualization

In [ ]:
# Show results
# 注意：如果路径不对，可能是因为训练还没结束或者名字变了
try:
    display(Image(filename=f"runs/train/pitch_yolo11x/results.png"))
except:
    print("Results image not found yet.")

In [ ]:
# Show predictions on validation set
try:
    display(Image(filename=f"runs/train/pitch_yolo11x/val_batch0_pred.jpg"))
except:
    print("Validation prediction image not found yet.")

## 5. Export Model

Download `best.pt` to use in your local Tactix project.

In [ ]:
import shutil
import os

source_path = "runs/train/pitch_yolo11x/weights/best.pt"
dest_path = "pitch_yolo11x_best.pt"

if os.path.exists(source_path):
    # Copy to working directory for easier download
    shutil.copy(source_path, dest_path)
    print(f"✅ Model saved as '{dest_path}'. Ready for use!")
else:
    print("❌ best.pt not found. Did the training finish?")